## Prerequisities

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import pipeline

In [ ]:
import random

## Source

In [ ]:
def get_pred_model():
    # model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
    # tokenizer = BertTokenizer.from_pretrained(
    #     "dkleczek/bert-base-polish-uncased-v1"
    # )
    model = BertForMaskedLM.from_pretrained("allegro/herbert-large-cased")
    tokenizer = BertTokenizer.from_pretrained(
        "allegro/herbert-large-cased"
    )
    pred_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    return pred_pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModel

def get_pred_model():
    # model = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
    # tokenizer = BertTokenizer.from_pretrained(
    #     "dkleczek/bert-base-polish-uncased-v1"
    # )
    model = BertForMaskedLM.from_pretrained("allegro/herbert-large-cased")
    # tokenizer = BertTokenizer.from_pretrained(
    #     "allegro/herbert-large-cased"
    # )
    tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
    pred_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)
    return pred_pipeline

In [ ]:
class WordGenerator:
    """
    Class to extend given sentence
    """

    def __init__(self):
        self.model = get_pred_model()

    def add_punctuation(self, user_input):
        user_input = user_input.strip()
        if user_input[-1] not in "?!.":
            user_input += "."
        return user_input

    def choose_sent(self, words, sequences_words, scores):
        chosen = random.choices(sequences_words, scores)[0]
        new_word = chosen[1]
        if new_word not in words:
            res = " ".join(chosen[0])
        return " ".join(words)

    def generate_in(self, user_input, start_words, add_word_num):
        user_input = self.add_punctuation(user_input)
        result = start_words

        for i in range(add_word_num):
            words = result.split()
            if len(words) == 1:
                mask_index = 1
            else:
                mask_index = random.randint(1, len(words) - 1)
            words.insert(mask_index, self.model.tokenizer.mask_token)
            preds = self.model(f"{user_input} {' '.join(words)}")
            sequences_words = [
                (pred["sequence"].split()[-len(words) - 1 : -1], pred["token_str"])
                for pred in preds
            ]
            scores = [pred["score"] for pred in preds]
            result = self.choose_sent(result.split(), sequences_words, scores)

        return result

In [ ]:
word_generator = WordGenerator()

In [ ]:
word_generator.generate_in("Lubię truskawki", "Truskawki owoce", 3)